In [1]:
#  http://www.ciids.cn/node_59109.htm

In [5]:
import json
import os 
import requests
import time
from datetime import datetime,timedelta
from datetime import date
import re
import sys
import urllib, urllib.request, urllib.parse
import random
from scrawl import *
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.firefox.options import Options

# Date and time
start_time = time.time()
current_time = datetime.now().strftime("%H-%M-%S")
created_on = date.today().strftime("%Y-%m-%d")

# client_id = sys.argv[1]
client_id = '5f69d22ef472d6646f577fa6'  # Europe
site = 'ciids_cn_publication'
site_name = 'Institute for Innovation and Development Strategy (China)'
c = Crawl()  # creating object

# create directories to store logs.
log_path = c.create_directories(project_path, client_id, site)

## creating pdf directories
pdf_directory = c.create_pdf_directories(project_path, site)
# create image directories
image_directory = c.create_image_directories(project_path)


# Driver Connection      
options = webdriver.ChromeOptions()
options.add_argument('--headless')
browser = webdriver.Chrome(f'{project_path}/chromedriver', options = options )
browser.maximize_window()

# options = Options()
# options.headless = True
# browser = webdriver.Firefox(options=options)
# browser.maximize_window()
# logger
logger = log_func(log_path, created_on, current_time)
logger.info("Process Started ...\n")

# initialize variables
skipped_due_to_headline = 0
skipped_due_to_content = 0
skipped_due_to_date = 0
missing_overall_tonality = 0
no_of_data = 0
duplicate_data = 0  
unable_to_fetch_article_url = 0
unable_to_fetch_cat_url = 0
# unable_to_download_pdf = 0
publish_source = 'ciids.cn'
country = 'China'
language = 'English'
images_path = []
foot_fall = c.get_foot_fall(publish_source)

home_url = 'http://www.ciids.cn/node_59109.htm'
try:
    browser.get(home_url )
except:
    logger.info('Unable to fetch home page\n')
    broswer.quit()
    exit()
time.sleep(3)
home_page = browser.page_source

home_page = c.scrap('Contact US</a></span></div>(.*?)<div\s*class="mainbody">',home_page)
for _ in home_page.split('href=')[1:-1]:
    
    cat_url = c.scrap('"(.*?)"', _) 
    if ('node_59119.htm' in cat_url) or ('content_14670956.htm' in cat_url) or ('content_14671320.htm' in cat_url) or ('content_14671236.htm' in cat_url) or ('59131.htm' in cat_url) or ('59462.htm' in cat_url) or ('59463.htm' in cat_url) or ('59461.htm' in cat_url):
        continue
    if ('59121.htm' in cat_url) or ('59117.htm' in cat_url):
        continue

    logger.info(f'Fetching cat url  {cat_url}\n')
    
    cat_page = c.download_page(cat_url) 
    
    if cat_page.startswith('Unable to fetch'):
        logger.info(cat_page) 
        unable_to_fetch_cat_url += 1
        continue    
    for i in cat_page.split('<li><p><a href=')[1:]:
        # source_link
        source_link = c.scrap('"(.*?)"', i)
        
        if 'http' not in source_link:
            source_link = 'http://www.ciids.cn/' +  source_link
        
        if ('14604524' in source_link) or ('14604523' in source_link) or ('14604522' in source_link) or ('14604521' in source_link) or('14604520' in source_link) or('14604519' in source_link) or('14604518' in source_link) or('14604517' in source_link) or('14604516' in source_link) or('14604515' in source_link) or('14604514' in source_link) or('14604513' in source_link) or('14604512' in source_link) or('14604511' in source_link) or('14604510' in source_link) or('14604509' in source_link) or('14604508' in source_link) or('14604507' in source_link):
            language = 'Chinese'
        else:
            language = "English"

        # handle duplicates
        source_link_query = {'source_link':source_link}
        dic = cl_data.find_one(source_link_query,{'source_link': 1}) 
        if dic:
            duplicate_data += 1
            continue

        time.sleep(random.randint(1,3))

        logger.info(f'Fetching {source_link}\n')
        
        page = c.download_page(source_link)   # here the page wil get download
        
        if page.startswith('Unable to fetch'):     
            logger.info(page) # writes error message with error code
            unable_to_fetch_article_url += 1
            continue    

        source_headline = c.scrap('<title>(.*?)-', page)
        source_headline = re.sub('&.*?;','',source_headline)
        
        # skip if headline not found
        if not source_headline:
            logger.info(f'Skipping due to headline {source_link}\n')
            skipped_due_to_headline += 1
            continue
            
         # Date and time
        pub_date, publish_time = '', ''
        publish_time = current_time
        try:
            date_time_str = c.scrap('publishdate".*?content="(.*?)"', page).strip()
            date_time_str = re.sub('[^\w+]', '', date_time_str)  
            date_time_obj = datetime.strptime(date_time_str, '%Y%m%d')
            ist_date_time = date_time_obj - timedelta(hours = 2,minutes = 30)  
            ist_date_time = ist_date_time.strftime('%Y-%m-%d %H:%M:%S')
            pub_date = ist_date_time[:10]
            publish_time = ist_date_time[11:]
            
        except:
            pass
        
        # skip null date
        if not pub_date:
            logger.info(f'Skipping due to date {source_link}\n')            
            skipped_due_to_date += 1
            continue

        # break if date is not today's date    # it wil  check and if the artcles mai pub_date match nhi hoga so upar loop ke bhar aa jayega,n fir first for loop mai jakar dursra category leyga
#         if pub_date != created_on:
#             break    


        # source_content         
        source_content= c.scrap('<div\s*class="cont">(.*?)<div\s*class="clear">',page)   
        source_content = re.sub('&.*?;','',source_content)
        source_content = c.strip_html(source_content)  

        if not source_content:
            logger.info(f'Skipping due to content {source_link}\n')            
            skipped_due_to_content += 1
            continue

        
        journalist =c.scrap("Author\s*:\s*(.*?)</span>",page)

        if not journalist: journalist = 'NA'

        # current date and time 00
        harvest_time = datetime.now().strftime("%H:%M:%S")

        # temp link
        temp_link = source_link

        # headline and content 
        headline = source_headline
        content = source_content

        # overall_tonality
        overall_tonality = ''

        # word count
        word_count = len((source_headline + ' ' + source_content).split())

        html_content = ''

        # image_urls
        image_urls = []
    


        # storing the above data in a dictionary
        clientdata ={
                        "client_master" : client_id, 
                        "articleid":client_id,
                        "medium":'Web' ,
                        "searchkeyword":[],
                        "entityname" : [] ,
                        "process_flage":"1",
                        "na_flage":"0",
                        "na_reason":"",
                        "qc_by":"",
                        "qc_on":"",
                        "location":"",
                        "spokeperson":"",
                        "quota":"",
                        "overall_topics":"",
                        "person":"",
                        "overall_entites":"",
                        "overall_tonality": overall_tonality,
                        "overall_wordcount":word_count,
                        "article_subjectivity":"",
                        "article_summary":"",
                        "pub_date":pub_date,
                        "publish_time":publish_time,
                        "harvest_time":harvest_time,
                        "temp_link":temp_link,
                        "publish_source": publish_source,
                        "programme":'null',
                        "feed_class":"News",
                        "publishing_platform":"",
                        "klout_score":"",
                        "journalist":journalist,
                        "headline":headline,
                        "content":content,
                        "source_headline":source_headline,
                        "source_content":source_content,
                        "language":language,
                        "presence":'null',
                        "clip_type":'null',
                        "prog_slot":'null',
                        "op_ed":'0',
                        "location_mention":'',
                        "source_link":source_link,
                        "author_contact":'',
                        "author_emailid":'',
                        "author_url":'',
                        "city":'',
                        "state":'',
                        "country":country,
                        "source":publish_source,
                        "foot_fall":foot_fall,
                        "created_on":created_on,
                        "active":'1',
                        'crawl_flag':2,
                        "images_path":images_path,
                        "html_content":html_content,
                      
                    }
        cl_data.insert_one(clientdata)  
        no_of_data += 1

browser.quit()
logger.info('Iteration complete\n')   
logger.info(f'Number of data: {no_of_data}\n')
logger.info(f'Duplicate data: {duplicate_data}\n')
logger.info(f'Unable to fetch cat url: {unable_to_fetch_cat_url}\n')
logger.info(f'Unable to fetch article url: {unable_to_fetch_article_url}\n')
logger.info(f'Skipped due to headline: {skipped_due_to_headline}\n')
logger.info(f'Skipped due to content: {skipped_due_to_content}\n')
logger.info(f'Skipped due to date: {skipped_due_to_date}\n')
logger.info(f'country: {country}\n')
logger.info(f'language: {language}\n')
logger.info(f'Processing finished in {time.time() - start_time} seconds.\n')